In [ ]:
%matplotlib inline
#use %matplotlib notebook if in Jupyter notebook (not lab) for more interactive plotting

# Preparation for programming
# Make sure to execute this cell first!
import os
from os import path
import pandas as pd
import glob
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
from obspy import Stream
from obspy.core import read, UTCDateTime
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.geodetics import kilometers2degrees, gps2dist_azimuth
from obspy.imaging.spectrogram import spectrogram
from obspy.clients.fdsn import Client
import matplotlib.pylab as plt
from matplotlib import dates 
from matlab2datetime import matlab2datetime
from plotw_rs import plotw_rs
import datetime

# Python adaptation of run_getwaveform_626.m written by Carl Tape for Applied Seismology,GEOS 626, 
# Python coding done by Nealey Sims
#
# Load waveforms from the IRIS database, then plot record sections.
#
# copied from run_getwaveform_short.m in GEOTOOLS
# calls homebrew function plotw_rs.py

#==========================================================================
# USER PARAMETERS: READ BEFORE RUNNING

iex = 1                 # CHANGE THIS [1,2,3,4,5,6,7_0,7_1,7_2] OR ADD YOU OWN EXAMPLE [8]
iprocess = False        # iprocess = True to deconvolve
                        # 'DISP' is default output. Can change to ACC or VEL below
bspectrogram = True     # 

wexample="examp"+str(iex)
directory = "./datawf/" + str(wexample)
if not path.exists(directory):  # If data directory doesn't exist, it will create one
    os.makedirs(directory)
drec=os.listdir(directory)
if len(drec) < 2:
    print("Waveform directory empty. Dowloading from IRIS")
    wvfrm_download = True
else:
    wvfrm_download = False

#==========================================================================

In [ ]:
spdy = 86400

# default parameters for plotting record sections
# note: you can over-ride these within each example below
rssort = 2      # =1 by azimuth, =2 by distance
iabs = 0
T1 = []
T2 = []
trshift = 0
tmark = []       # USE brackets if input is not single argument 
pmax = 50        # otherwise, single scalar is accepted without brackets
iintp = 0
inorm = 1
nfac = 1
azstart = []
iunit = 1
tlims = []     # time limits for plotting
imap = 1
# List of preffered networks
networks="AK,AT,AU,AV,BK,CI,CN,CU,GT,IC,II,IM,IU,MS,TA,TS,US,XE,XM,XR,YM,YV,XF,XP,XZ"
#==========================================================================

# default parameters for waveform extraction
samplerate = []
cutoff = []

if iex == 1:
    #Yahtse event recorded within two databases (AEC, Yahtsee
    stasub = [0 ,200];

    # source parameters (some can be empty)
    #originTime = ('2010/09/18 14:15:02')
    originTime = dates.date2num(datetime.datetime(2010,9,18,14,15,2))
    elat = 60.155496
    elon = -141.378343
    edep_km = 0
    eid = []
    mag = []

    chan = ["HHZ","BHZ"]

    duration_s = 70
    oshift = 20
    T1 = 0.1
    T2 = 2
    bspectrogram = True
elif iex == 2:
    # Mw 7.5 SE Alaska
    ## can take 17:44.837403s to run first time for download
    stasub = [0, 500];
    # source parameters (some can be empty)
    #originTime = ('2013/01/05 08:58:32.4')
    originTime = dates.date2num(datetime.datetime(2013,1,5,8,58,32,4))
    elat = 55.62
    elon = -135.13
    edep_km = 0
    eid = []
    mag = []
    # broadband channels
    chan1 = ['BHZ','BHE','BHN','BH1','BH2']
    # strong motion channels
    chan2 = ['BNZ','BNE','BNN','BN1','BN2','BLZ','BLE','BLN','BL1','BL2',
            'HNZ','HNE','HNN','HN1','HN2','HLZ','HLE','HLN','HL1','HL2']
    # warning: waveforms will have different units (nm/s, nm/s^2)
    chan = chan1+ chan2
    
    duration_s = 300
    oshift = 50
elif iex == 3:
    # explosion in Fairbanks
    # source parameters (some can be empty)
    #originTime = ('2013/02/03 01:10:31.495')
    originTime = dates.date2num(datetime.datetime(2013,2,3,1,10,31,495))
    #elat = 64.8156; elon = -147.9419;  # original AEC
    #elat = 64.8045; elon = -147.9653;  # reviewed AEC
    elat = 64.80175; elon = -147.98236 # infrasound
    edep_km = 0
    eid = []
    mag = []
    # broadband channels
    chan = ['SHZ','HHZ','BHZ']
    stasub = [0, 200]
    duration_s = stasub[1]/0.30  # air wave
    oshift = 50
    # record section
    T1 = [0.2]
    T2 = [1]
elif iex == 4:
    # very low frequency (VLF) event near Kantishna
    # source parameters (some can be empty)
    #originTime = ('2014/01/22 12:14:34');
    originTime = dates.date2num(datetime.datetime(2014,1,22,12,14,34))
    elat = 63.463; elon = -150.109
    edep_km = 38
    eid = []
    mag = 1.7
    # broadband channels
    chan = ['BHZ']
    stasub = [0, 200]
    duration_s = 100 
    oshift = 0
    T1 = []; T2 = [2]
    
elif iex == 5:
    # landslide near Lituya
    stasub = [0 ,1000];
    # source parameters (some can be empty)
    #originTime = ('2014/02/16 14:24:00')
    originTime = dates.date2num(datetime.datetime(2014,2,16,14,24,0))
    elat = 58.68; elon = -137.37
    edep_km = 0
    eid = []
    mag = []
    chan = ["BHZ"]  # consider HHZ as well
    duration_s = 600
    oshift = 0
    T1 = [10]; T2 = [40]
    pmax = 40
    
elif iex == 6:
    # Sumatra Mw 8.6 in Alaska and triggered earthquakes
    # four different events
    #originTimeS = ('2012/04/11 08:38:37.30');    # Sumatra (CMT-PDE)
    originTimeS = dates.date2num(datetime.datetime(2012,4,11,8,38,37,30))
    
    # triggered events (plot in next example)
    #originTimeA = ('2012/04/11 09:00:09.71');    # Andreanof (NEIC)
    originTimeA = dates.date2num(datetime.datetime(2012,4,11,9,0,9,71))
    #originTimeN = ('2012/04/11 09:21:57.44');    # Nenana (NEIC)
    originTimeN = dates.date2num(datetime.datetime(2012,4,11,9,21,57,44))
    #originTimeI = ('2012/04/11 09:40:58.02');    # Iliamna slab (NEIC)
    originTimeI = dates.date2num(datetime.datetime(2012,4,11,9,40,58,2))
    
    elatS = 2.24; elonS = 92.78; edepS = 40.03; emagS = 8.6
    elatA = 51.36; elonA = -176.10; edepA = 20; emagA = 5.5
    elatN = 64.9222; elonN = -148.9461; edepN = 19.4; emagN = 3.88
    elatI = 60.10; elonI = -152.83; edepI = 101; emagI = 2.9
    
    # source
    elat = elatS; elon = elonS; edep_km = edepS; mag = emagS
    eid = '201204110838A';  # GCMT
    originTime = originTimeS;
    chan = ['BHZ']
    dmax_deg = 25
    stasub = [elonN, elatN, 0, dmax_deg, 0, 360]
    # parameters for Sumatra waveforms across Alaska
    duration_s = 3600*2.0
    oshift = 3600*0.25;
    T1 = [1/4]; T2 = [1/2]     # P wave + triggered events
    #T1 = [2]; T2 = [1000];     % full wavetrain (no triggered events visible)
    iunit = 2
    # three different triggered events are visible
    tmark = [UTCDateTime(dates.num2date(originTimeA)),
             UTCDateTime(dates.num2date(originTimeN)),
             UTCDateTime(dates.num2date(originTimeI))]
    
elif iex == 7_0:
    # one of the triggered earthquakes
    #originTimeA = ('2012/04/11 09:00:09.71');    # Andreanof (NEIC)
    originTimeA = dates.date2num(datetime.datetime(2012,4,11,9,0,9,71))
    elatA = 51.36; elonA = -176.10; edepA = 20; emagA = 5.5
    originTime = originTimeA; elat = elatA; elon = elonA; edep_km = edepA; # Andreanof
    mag = emagA; stasub = [0 , 2000]; duration_s = 600;
    eid = []
    chan = ['BHZ']
    # bandpass
    oshift = 10; T1 = [1/4]; T2 = [1/2]
    
elif iex == 7_1:
    # one of the triggered earthquakes
    #originTimeN = ('2012/04/11 09:21:57.44');    # Nenana (NEIC)
    originTimeN = dates.date2num(datetime.datetime(2012,4,11,9,21,57,44))
    elatN = 64.9222; elonN = -148.9461; edepN = 19.4; emagN = 3.88
    originTime = originTimeN; elat = elatN; elon = elonN; edep_km = edepN; # Nenana
    mag = emagN; stasub = [0 , 200]; duration_s = 200;   
    chan = ['BHZ']
    # bandpass
    oshift = 10; T1 = [1/4]; T2 = [1/2]
    
elif iex == 7_2:
    # one of the triggered earthquakes
    #originTimeI = ('2012/04/11 09:40:58.02');    # Iliamna slab (NEIC)
    originTimeI = dates.date2num(datetime.datetime(2012,4,11,9,40,58,2))
    elatI = 60.10; elonI = -152.83; edepI = 101; emagI = 2.9
    originTime = originTimeI; elat = elatI; elon = elonI; edep_km = edepI; # Iliamna
    mag = emagI; stasub = [0 , 400]; duration_s = 200;  
    chan = ['BHZ']
    # bandpass
    oshift = 10; T1 = [1/4]; T2 = [1/2]
    
elif iex == 8: 
    print(" ")
    # TRY YOUR OWN EXAMPLE HERE
    
# tshift (and trshift) are for plotting record sections only
tshift = [oshift + trshift]

#startTime = originTime - max(oshift)/spdy;
startTime = originTime - oshift/spdy
endTime   = originTime + duration_s/spdy
dur_dy = endTime-startTime
print('origin time is %s\n' % (dates.num2date(originTime)))
print('startTime is %s\n' % (dates.num2date(startTime)))
print('total length of time requested: %.2f s (= %.2f min = %.2f hours)\n' %
    (dur_dy*spdy,dur_dy*3600,dur_dy*24))

# additional user parameters
##iint = 0            # integrate waveforms: =1 for displacement, =0 for velocity
iprocess = False        # iprocess = True to deconvolve

# max radius for station search
if len(stasub)==2:
    minr=kilometers2degrees(stasub[0])
    maxr=kilometers2degrees(stasub[1])
    minlat=None
    maxlat=None
    minlong=None
    maxlong=None
    llong=elon
    llat=elat
elif len(stasub)==4:
    minr=None
    maxr=None
    minlat=stasub[2]
    maxlat=stasub[3]
    minlong=stasub[0]
    maxlong=stasub[1]
elif len(stasub)==6:
    minr=stasub[2]
    maxr=stasub[3]
    minlat=None
    maxlat=None
    minlong=None
    maxlong=None
    llong=stasub[0]
    llat=stasub[1]
    minaz=stasub[4]
    maxaz=stasub[5]

In [ ]:
# Setting up parameters for getting waveforms
client = Client("IRIS")
starttime = originTime-oshift/spdy  
endtime = originTime+duration_s/spdy

t = UTCDateTime(dates.num2date(originTime))    
starttime = UTCDateTime(dates.num2date(starttime))
endtime = UTCDateTime(dates.num2date(endtime))
# Measure download speed
cstart = datetime.datetime.now()

#Initialize variables for grabbing waveforms
inventory = Inventory()
ntwk=[]
st = []
slat=[]
slon=[]
stanames=[]
cha=[]
stas=[]
ST=Stream()

for chan_code in chan: 
    try:
        inventory += client.get_stations(minlatitude=minlat, maxlatitude=maxlat, 
                    minlongitude=minlong, maxlongitude=maxlong, longitude=llong, 
                    latitude=llat, minradius=minr, maxradius=maxr, 
                    network="AK,AT,AU,AV,BK,CI,CN,CU,GT,IC,II,IM,IU,MS,TA,TS,US,XE,XM,XR,YM,YV,XF,XP,XZ",
                    starttime=starttime, endtime =endtime,  
                    channel=chan_code, level="response")
    except:
        #print("No data for channel %s " % (chan_code))
        pass
print(inventory)
#inventory.write("example_inv.txt",format="STATIONTXT")
for net in inventory:     
    for sta in net:
        if sta.code not in stanames:
        # Use inventory to save station locations
            ntwk.append(net.code)
            stanames.append(sta.code)
            slat.append(sta.latitude)
            slon.append(sta.longitude)
            
if wvfrm_download == True:
    for ii,sta in enumerate(stanames):
        for chan_code in chan:
            st=[]
            try:
                st = client.get_waveforms(ntwk[ii], stanames[ii], "*", chan_code, starttime, endtime)
            except:
                pass       
            if len(st)!=0:
                try:
                    st.merge(method=1, fill_value=0)              # merge the traces
                except:
                    print("couldn't merge traces")
                    pass
                if len(stasub)==6:
                    azim=gps2dist_azimuth(llat, llong, slat[ii], slon[ii])
                    if azim[1]>minaz and azim[1]<maxaz:
                        st.write(path.join(directory, stanames[ii]+chan_code), format = 'SAC')
                    else:
                        print("azimuth out of range",azim[1])
                else:    
                    st.write(path.join(directory, stanames[ii]+chan_code), format = 'SAC')  
                
# Print download time
cend = datetime.datetime.now() - cstart
print ("start data download: ", cend, " s")

# Read saved sac files containing waveform data
st1 = read(directory+"/*",header=None)

if iprocess == True:
    # Deconvolve instrument response using obspy remove_response
    st1.remove_response(inventory=inventory, water_level=60)
    
# Add station location data to the Obspy Trace
for i, tr in enumerate(st1):
    for s, stan in enumerate(stanames):
        if str(tr.stats.station) == str(stan) and str(tr.stats.network) == str(ntwk[s]):
            tr.stats.sampling_rate=round(tr.stats.sampling_rate)      # rounds samp rate for merging purposes
            tr.stats.sac.stla=slat[s]                                 # attach station location to trace
            tr.stats.sac.stlo=slon[s]
            ST.append(tr)
ST.merge(method=1, fill_value=0)

# Remove instrument response. Can change output to "ACC" or "VEL"
if iprocess==True:
    ST.remove_response(inventory=inventory,water_level=60,output="DISP")

In [ ]:
# plot record section (note: the time this function takes is printed from plotw_rs)
w = plotw_rs(ST,elat,elon,rssort,iabs,tshift,tmark,T1,T2,pmax,iintp,inorm,tlims,
                 nfac,azstart,iunit,imap)

In [ ]:
if bspectrogram == True:
    # Plot Spectrogram of a single station waveform
    station_nm="BOOM"
    for tr in ST:
        if str(tr.stats.station) == station_nm:
            dta=tr.data
            npts=tr.stats.npts
            samp=tr.stats.sampling_rate
            stanam=tr.stats.station
            chan=tr.stats.channel
            stime=tr.stats.starttime
            TR=tr
    
    fig = plt.figure(figsize=(12,10))     # Can change numbers to alter size of plot
    ax1 = fig.add_axes([0.1, 0.75, 0.7, 0.2]) #[left bottom width height]
    ax2 = fig.add_axes([0.1, 0.1, 0.7, 0.60], sharex=ax1)
    ax3 = fig.add_axes([0.83, 0.1, 0.03, 0.6])

    #plot waveform (top subfigure)    
    t = np.arange(npts) / samp
    ax1.plot(t, dta, 'b', linewidth=0.7)

    #plot spectrogram (bottom subfigure)
    ax2.specgram(TR, Fs=samp, noverlap= 0.8*256, vmin=-10, vmax=90, cmap="jet")
    ax2.set_xlabel('Time - Seconds')
    ax2.set_ylabel('Frequency (Hz)')
    mappable = ax2.images[0]
    plt.colorbar(mappable=mappable, cax=ax3)
    plt.ylabel('Relative Amplitude (dB)')
    fig.suptitle(str(stanam)+" ("+ str(chan) + ") " + str(stime))